**IMPORT THE LIBRARY**

In [ ]:
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import plotly.graph_objects as go
plt.style.use ('fivethirtyeight')
import plotly.express as px

In [ ]:
#!pip install pandas_datareader

In [ ]:
#Reading data set and setting the index as Date
df = pd.read_csv('read.csv.csv')
df = df.set_index('Date')

In [ ]:
#Droping null values
df=df.dropna()

In [ ]:
#Showing the graph High, Low, Open, Close in Our dataset. x=Date y=INR
fig = go.Figure()
fig.add_trace(go.Scatter(x = df.index, y = df.High,mode='lines',name='High',marker_color = '#2CA02C',visible = "legendonly"))
fig.add_trace(go.Scatter(x = df.index, y = df.Low,mode='lines',name='Low',marker_color = '#D62728',visible = "legendonly"))
fig.add_trace(go.Scatter(x = df.index, y = df.Open,mode='lines',name='Open',marker_color = '#FF7F0E',visible = "legendonly"))
fig.add_trace(go.Scatter(x = df.index, y = df.Close,mode='lines',name='Close',marker_color = '#1F77B4'))

fig.update_layout(title='Closing price history',titlefont_size = 28,
                  xaxis = dict(title='Date',titlefont_size=16,tickfont_size=14),height = 800,
                  yaxis=dict(title='Price in INR (₹)',titlefont_size=16,tickfont_size=14),
                  legend=dict(y=0,x=1.0,bgcolor='rgba(255, 255, 255, 0)',bordercolor='rgba(255, 255, 255, 0)'))
fig.show()

# Preprocessing

**DATA SPLITTING**

In [ ]:
data = df.filter(['Close'])
dataset = data.values
training_data_len = math.ceil(len(dataset) * .8)

In [ ]:
training_data_len

292

**NORMALIZATION**

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)
scaled_data

array([[0.00000000e+00],
       [1.68215282e-03],
       [2.55033356e-03],
       [3.81151792e-03],
       [4.93452445e-04],
       [1.18260237e-03],
       [3.58185117e-03],
       [8.53668979e-03],
       [1.18752753e-02],
       [2.92551827e-02],
       [3.19789916e-02],
       [3.13209864e-02],
       [3.47022018e-02],
       [3.32294390e-02],
       [3.40736579e-02],
       [4.51635432e-02],
       [3.78286476e-02],
       [4.10197399e-02],
       [4.31006763e-02],
       [4.75918791e-02],
       [4.67512704e-02],
       [4.31766583e-02],
       [5.08348418e-02],
       [5.43478940e-02],
       [8.20479907e-02],
       [8.17804452e-02],
       [6.77569309e-02],
       [8.01271551e-02],
       [7.73066199e-02],
       [7.65133736e-02],
       [8.43750922e-02],
       [9.53889956e-02],
       [9.61832333e-02],
       [9.14009347e-02],
       [8.92450825e-02],
       [1.03812517e-01],
       [1.21612682e-01],
       [1.24650576e-01],
       [1.24901193e-01],
       [1.40305700e-01],


In [ ]:
train_data = scaled_data[0:training_data_len, :]
x_train = []
y_train = []
for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])

In [ ]:
test_data=scaled_data[training_data_len:len(scaled_data),:1]

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

(232, 60, 1)

**LSTM(MODEL BUILDING)**

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences = False))
model.add(Dense(25))
model.add(Dense(1))
model.compile(optimizer = 'adam', loss = 'mean_squared_error') #dog=input output=cat
model.fit(x_train, y_train, batch_size = 5, epochs = 100,verbose=1)

Epoch 1/100
47/47 [==============================] - 7s 54ms/step - loss: 0.0337
Epoch 2/100
47/47 [==============================] - 3s 54ms/step - loss: 0.0083
Epoch 3/100
47/47 [==============================] - 3s 53ms/step - loss: 0.0074
Epoch 4/100
47/47 [==============================] - 2s 53ms/step - loss: 0.0072
Epoch 5/100
47/47 [==============================] - 3s 55ms/step - loss: 0.0068
Epoch 6/100
47/47 [==============================] - 3s 54ms/step - loss: 0.0054
Epoch 7/100
47/47 [==============================] - 3s 54ms/step - loss: 0.0049
Epoch 8/100
47/47 [==============================] - 3s 55ms/step - loss: 0.0053
Epoch 9/100
47/47 [==============================] - 3s 55ms/step - loss: 0.0039
Epoch 10/100
47/47 [==============================] - 3s 56ms/step - loss: 0.0036
Epoch 11/100
47/47 [==============================] - 3s 57ms/step - loss: 0.0032
Epoch 12/100
47/47 [==============================] - 3s 57ms/step - loss: 0.0028
Epoch 13/100
47/47 [=====

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 60, 50)            10400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense (Dense)                (None, 25)                1275      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 26        
Total params: 31,901
Trainable params: 31,901
Non-trainable params: 0
_________________________________________________________________


**TRAINING AND TESTING THE MODEL**

In [ ]:
test_data = scaled_data[training_data_len - 60: , :]
x_test = []
y_test = dataset[training_data_len:, :]

for i in range (60, len(test_data)):
    x_test.append(test_data[i - 60:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

rsme = np.sqrt(np.mean(predictions - y_test) ** 2)
rsme

1066.462565048612

In [ ]:
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = train.index, y = train.Close,mode='lines',name='Close',marker_color = '#1F77B4'))
fig.add_trace(go.Scatter(x = valid.index, y = valid.Close,mode='lines',name='Val',marker_color = '#FF7F0E'))
fig.add_trace(go.Scatter(x = valid.index, y = valid.Predictions,mode='lines',name='Predictions',marker_color = '#2CA02C'))

fig.update_layout(title='Model',titlefont_size = 28,hovermode = 'x',
                  xaxis = dict(title='Date',titlefont_size=16,tickfont_size=14),height = 800,
                  yaxis=dict(title='Close price in INR (₹)',titlefont_size=16,tickfont_size=14),
                  legend=dict(y=0,x=1.0,bgcolor='rgba(255, 255, 255, 0)',bordercolor='rgba(255, 255, 255, 0)'))
fig.show()

In [ ]:
len(test_data)

132

In [ ]:
x_input=test_data[32:].reshape(1,-1)
x_input.shape

(1, 100)

In [ ]:
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

In [ ]:
temp_input

[0.44768216335882016,
 0.4595432785321818,
 0.4300113284545336,
 0.43938153762653864,
 0.4321053775014665,
 0.4133731148112858,
 0.4310075112971087,
 0.4256922915120698,
 0.4394771603945318,
 0.41870718977274735,
 0.41001445827927874,
 0.4123191692267042,
 0.39236752907177896,
 0.3854874028903994,
 0.38762368905032063,
 0.39267553467491545,
 0.3739237059662416,
 0.35456839877297175,
 0.3986877980995204,
 0.40256489770206283,
 0.42686131363785895,
 0.4404781318582057,
 0.4607386380740154,
 0.4988052577401578,
 0.5384436674646254,
 0.5497249853787755,
 0.5499647157225432,
 0.5926241467828315,
 0.5809523317968661,
 0.5493225167350582,
 0.5345170730000299,
 0.5144246922637516,
 0.5449669665567146,
 0.566459270751976,
 0.6037520043504114,
 0.6370674140199069,
 0.6225543754607352,
 0.6717293989212432,
 0.6567817929781844,
 0.6569466262998646,
 0.6346249928071774,
 0.6990804313771233,
 0.6857417273059068,
 0.6847851254048144,
 0.6648162013158214,
 0.639740586930998,
 0.6417676700999612,
 0.67

**BITCOIN PRICE PREDICTION FOR FUTURE 30 DAYS**

In [ ]:
from numpy import array

lst_output=[]
n_steps=100
i=0
while(i<30):
    
    if(len(temp_input)>100):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1 

print(lst_output)

[0.8540225]
101
1 day input [0.45954328 0.43001133 0.43938154 0.43210538 0.41337311 0.43100751
 0.42569229 0.43947716 0.41870719 0.41001446 0.41231917 0.39236753
 0.3854874  0.38762369 0.39267553 0.37392371 0.3545684  0.3986878
 0.4025649  0.42686131 0.44047813 0.46073864 0.49880526 0.53844367
 0.54972499 0.54996472 0.59262415 0.58095233 0.54932252 0.53451707
 0.51442469 0.54496697 0.56645927 0.603752   0.63706741 0.62255438
 0.6717294  0.65678179 0.65694663 0.63462499 0.69908043 0.68574173
 0.68478513 0.6648162  0.63974059 0.64176767 0.67847514 0.7286905
 0.72038351 0.72835484 0.73265495 0.6974101  0.72144269 0.68277802
 0.72331751 0.72032433 0.71893428 0.68493799 0.6870776  0.71926365
 0.72847112 0.74183429 0.74028756 0.77493391 0.79179225 0.68026706
 0.66648085 0.67222779 0.64335218 0.64943463 0.6659422  0.64486853
 0.68565643 0.70641712 0.69888963 0.68900899 0.70837118 0.68886915
 0.60427494 0.5630904  0.61827125 0.64356647 0.60419735 0.60183835
 0.61126131 0.59262766 0.56961954 0.

In [ ]:
print("Bitcoin price of future 30 days")
scaler.inverse_transform(lst_output)

Bitcoin price of future 30 days


array([[55882.39586937],
       [56448.42340899],
       [56964.58458768],
       [57401.54149272],
       [57762.93802482],
       [58067.85565202],
       [58345.43624454],
       [58629.16572499],
       [58951.39110255],
       [59340.27092325],
       [59817.51300201],
       [60398.20327441],
       [61090.90848564],
       [61897.76332049],
       [62815.18611396],
       [63834.07800556],
       [64940.05009963],
       [66113.51059532],
       [67330.6730051 ],
       [68565.52280305],
       [69790.43355846],
       [70979.17914401],
       [72108.68255923],
       [73161.36221664],
       [74126.65671661],
       [75001.37336733],
       [75788.67374276],
       [76497.11525273],
       [77139.79229018],
       [77730.87592579]])